# Urban Morphology with Python: City Structure as Predictor and Target

* [**Martin Fleischmann, PhD**](https://github.com/martinfleis)
  * [Research Associate](https://martinfleischmann.net) @ the Charles University in Prague
* [**James D. Gaboardi, PhD**](https://github.com/jGaboardi)
  * [Associate R&D Scientist](https://www.ornl.gov/staff-profile/james-d-gaboardi) @ Oak Ridge National Laboratory

[This workshop](https://github.com/martinfleis/sdss2025) walks you through assessment of urban form through urban morphometrics - quantitative characterisation of individual elements of urban form and their distribution in space. 

## Outline

- Data retrieval
- Data preprocessing
    - Removing overlaps from building data
    - Simplification of street networks
- Measuring urban form
    - Simple measurements
    - Spatial graphs and their use
    - Contextualisation
- Urban form as a target - clustering
- Urban form as a predictor - regression

## Environment setup

For instructions on how to setup the environment, please see the [README.md](https://github.com/martinfleis/sdss2025/blob/main/README.md).

In [ ]:
import geopandas
import geoplanar
import libpysal
import matplotlib.pyplot as plt
import momepy
import neatnet
import osmnx
import pandas
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import show
from clustergram import Clustergram
from sklearn.linear_model import LinearRegression

output_notebook()

Pick a place, ideally a town with a good coverage in OpenStreetMap and its local CRS.

In [ ]:
place = "Bath, England"
local_crs = 27700

We can interactively explore the place we just selected.

In [ ]:
place_geom = geopandas.tools.geocode(
    place, provider="nominatim", user_agent="sdss2025-workshop"
)
place_geom.explore()

## Input data

We can use ``OSMnx`` to quickly download data from OpenStreetMap.

### Buildings

In [ ]:
buildings = osmnx.features_from_place(place, tags={"building": True})
buildings.head()

The OSM input may need a bit of cleaning to ensure only proper polygons are kept.

In [ ]:
buildings.geom_type.value_counts()

In [ ]:
buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)

And we should re-project the data from WGS84 to the local projection in meters (momepy default values assume meters not feet or degrees). We will also drop unnecessary columns.

In [ ]:
buildings = buildings[["geometry"]].to_crs(local_crs)
buildings.head()

#### Topology

Buildings shall form a valid polygonal coverage - neighbouring buildings are edge-matched and there are no overlaps. Ideally, there should be no tiny (sliver) gaps between the polygons either but fixing that is beyond the scope of this workshop. Today, we will focus on overlaps only.

An easy check if it is the case can be done via `geoplanar`, a package aimed at planarity enforcment and validation.

In [ ]:
i, j = geoplanar.overlaps(buildings.geometry)

We can locate the buildings violating the coverage expectations.

In [ ]:
buildings.iloc[i].explore(tiles="cartodb positron", max_zoom=50, prefer_canvas=True)

#### Coverage enforcement

First, we can trip overlaps. By default, the largest polygon is trimmed.

In [ ]:
buildings = geoplanar.trim_overlaps(buildings, strategy="largest")

Let's see if there are any remaining.

In [ ]:
geoplanar.overlaps(buildings.geometry).shape

Try another run, this time trimming the smallest.

In [ ]:
buildings = geoplanar.trim_overlaps(buildings, strategy="smallest")

And the outcome.

In [ ]:
geoplanar.overlaps(buildings.geometry).shape

Much better. `momepy` is relatively robust to small imprecisions, so we can go ahead.

### Streets

Similar operations are done with streets.

In [ ]:
osm_graph = osmnx.graph_from_place(place, network_type="drive")
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)
streets = osmnx.graph_to_gdfs(
    osmnx.convert.to_undirected(osm_graph),
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True,
).reset_index(drop=True)
streets.head()

Streets pre-processing is trickier as there are a lot of complicated cases that need to be simplified to turn transportation network into a morphological one. Luckliy, the `neatnet` package should solve this automagically for us. But first, check the current state, together with `neatnet`'s interpretation of what needs fixing.

In [ ]:
ax = streets.explore(tiles="cartodb positron")
neatnet.get_artifacts(streets, threshold_fallback=7, exclusion_mask=buildings.geometry)[
    0
].explore(m=ax, color="red")

We can let `neatnet` fix these.

In [ ]:
streets_neat = neatnet.neatify(streets, exclusion_mask=buildings.geometry)

And check the outcome.

In [ ]:
m = streets.explore(tiles="cartodb positron", color="red")
streets_neat.explore(m=m)

In [ ]:
streets = streets_neat[["geometry"]].copy()

## Tessellation

We still lack a spatial unit, so let's generate morphological tessellation.

In [ ]:
limit = momepy.buffered_limit(buildings, 100)
tessellation = momepy.morphological_tessellation(buildings, clip=limit)

OpenStreetMap data are often problematic due to low quality of some polygons. If some collapse, we get a mismatch between the length of buildings and the length of polygons.

In [ ]:
collapsed, _ = momepy.verify_tessellation(tessellation, buildings)

Better to drop affected buildings and re-create tessellation.

In [ ]:
buildings = buildings.drop(collapsed).copy()
tessellation = momepy.morphological_tessellation(buildings, clip=limit)

The outcome may be a bit too heavy for an interactive visualisation, but let's try.

In [ ]:
tessellation.geometry.explore(prefer_canvas=True)

### Link streets

Link unique IDs of streets to buildings and tessellation cells based on the nearest neighbor join.

In [ ]:
buildings["street_index"] = momepy.get_nearest_street(
    buildings, streets, max_distance=100
)
buildings

Aattach the network index to the tessellation as well.

In [ ]:
tessellation["street_index"] = buildings["street_index"]

## Measure

Measure individual morphometric characters. For details see the User Guide and the API reference.

### Dimensions

In [ ]:
buildings["building_area"] = buildings.area
tessellation["tess_area"] = tessellation.area
streets["length"] = streets.length

### Shape

In [ ]:
buildings["eri"] = momepy.equivalent_rectangular_index(buildings)
buildings["elongation"] = momepy.elongation(buildings)
tessellation["convexity"] = momepy.convexity(tessellation)
streets["linearity"] = momepy.linearity(streets)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("eri", ax=ax[0], scheme="natural_breaks", legend=True)
buildings.plot("elongation", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

tessellation.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
streets.plot("linearity", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

### Spatial distribution

Measurements of spatial distribution take into account the relations between geometries.

In [ ]:
buildings["shared_walls"] = momepy.shared_walls(buildings) / buildings.length
buildings.plot(
    "shared_walls", figsize=(12, 12), scheme="natural_breaks", legend=True
).set_axis_off()

In some cases, those relations need to be identified first. Therefore, we need to build a spatial graph (e.g. contiguity) that captures them. Generate spatial graph using `libpysal`.

In [ ]:
queen_1 = libpysal.graph.Graph.build_contiguity(tessellation, rook=False)

You can check how the resulting graph looks like. Even though we have generated it based on tessellation, you can explore it with buildings as the two share the index the graph is encoded by.

In [ ]:
m = buildings.explore(tiles="cartodb positron", prefer_canvas=True)
queen_1.explore(buildings, m=m)

This allows us to measure variables where relation beyond intersection plays a role.

In [ ]:
tessellation["neighbors"] = momepy.neighbors(tessellation, queen_1, weighted=True)
tessellation["covered_area"] = queen_1.describe(tessellation.area)["sum"]
buildings["neighbor_distance"] = momepy.neighbor_distance(buildings, queen_1)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("neighbor_distance", ax=ax[0], scheme="natural_breaks", legend=True)
tessellation.plot("covered_area", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

To capture a wider context, you can define it as higher order contiguity (or KNN, distance...).

In [ ]:
queen_3 = queen_1.higher_order(3)
buildings_q1 = libpysal.graph.Graph.build_contiguity(buildings, rook=False)

We have generated higher order contiguity and Queen contiguity on buidings. Check how it differs from the one generated on tessellation.

In [ ]:
m = buildings.explore(tiles="cartodb positron", prefer_canvas=True)
buildings_q1.explore(buildings, m=m)

Combining the two graphs, we can measure more complex spatial relations.

In [ ]:
buildings["interbuilding_distance"] = momepy.mean_interbuilding_distance(
    buildings, queen_1, queen_3
)
buildings["adjacency"] = momepy.building_adjacency(buildings_q1, queen_3)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("interbuilding_distance", ax=ax[0], scheme="natural_breaks", legend=True)
buildings.plot("adjacency", ax=ax[1], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()

Considering streets and buildings together allows us to measure street profile.

In [ ]:
profile = momepy.street_profile(streets, buildings)
streets[profile.columns] = profile

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(24, 12))

streets.plot("width", ax=ax[0], scheme="natural_breaks", legend=True)
streets.plot("width_deviation", ax=ax[1], scheme="natural_breaks", legend=True)
streets.plot("openness", ax=ax[2], scheme="natural_breaks", legend=True)

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

### Intensity

Density of development can be measured without `momepy` as a simple ratio of area of buildings to that of tessellation cells.

In [ ]:
tessellation["car"] = buildings.area / tessellation.area
tessellation.plot("car", figsize=(12, 12), vmin=0, vmax=1, legend=True).set_axis_off()

### Connectivity

Another way of looking at urban form is through its street network.

In [ ]:
graph = momepy.gdf_to_nx(streets)
graph = momepy.node_degree(graph)
graph = momepy.closeness_centrality(graph, radius=400, distance="mm_len")
graph = momepy.meshedness(graph, radius=400, distance="mm_len")
nodes, edges = momepy.nx_to_gdf(graph)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(24, 12))

nodes.plot("degree", ax=ax[0], scheme="natural_breaks", legend=True, markersize=1)
nodes.plot(
    "closeness",
    ax=ax[1],
    scheme="natural_breaks",
    legend=True,
    markersize=1,
    legend_kwds={"fmt": "{:.6f}"},
)
nodes.plot("meshedness", ax=ax[2], scheme="natural_breaks", legend=True, markersize=1)

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

Link the streets and nodes to buildings to transfer the values.

In [ ]:
buildings["edge_index"] = momepy.get_nearest_street(buildings, edges)
buildings["node_index"] = momepy.get_nearest_node(
    buildings, nodes, edges, buildings["edge_index"]
)

Link all data together (to tessellation cells or buildings).

In [ ]:
buildings[tessellation.columns.drop(["geometry", "street_index"])] = tessellation.drop(
    columns=["geometry", "street_index"]
)
merged = buildings.merge(
    edges.drop(columns="geometry"),
    left_on="edge_index",
    right_index=True,
    how="left",
)
merged = merged.merge(
    nodes.drop(columns=["geometry", "x", "y"]),
    left_on="node_index",
    right_index=True,
    how="left",
)

## Understanding the context

Measure first, second and third quartile of distribution of values within an area around each building.

In [ ]:
percentiles = []
for column in merged.columns.drop(
    [
        "street_index",
        "node_index",
        "edge_index",
        "nodeID",
        "mm_len",
        "node_start",
        "node_end",
        "geometry",
    ]
):
    perc = momepy.percentile(merged[column], queen_3)
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)

Merge all together.

In [ ]:
percentiles_joined = pandas.concat(percentiles, axis=1)
percentiles_joined.head()

See the difference between original convexity and spatially lagged one.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

buildings.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
merged.plot(
    percentiles_joined["convexity_50"].values,
    ax=ax[1],
    scheme="natural_breaks",
    legend=True,
)

ax[0].set_axis_off()
ax[1].set_axis_off()

## Clustering

Now we can use obtained values within a cluster analysis that should detect types of urban structure.

Standardize values before clustering.

In [ ]:
standardized = (
    percentiles_joined - percentiles_joined.mean()
) / percentiles_joined.std()
standardized.head()

### How many clusters?

To determine how many clusters we should aim for, we can use a little package called `clustergram`. See its [documentation](https://clustergram.readthedocs.io) for details.

In [ ]:
cgram = Clustergram(range(1, 12), random_state=456, n_init=10)
cgram.fit(standardized.fillna(0))

show(cgram.bokeh())

Clustegram gives us also the final labels. (Normally, you would run the final clustering on much larger number of initialisations.)

In [ ]:
cgram.labels.head()

The first stable split seems to be 5.

In [ ]:
merged["cluster"] = cgram.labels[5].values
buildings["cluster"] = merged["cluster"]
buildings.plot(
    "cluster", categorical=True, figsize=(16, 16), legend=True
).set_axis_off()

We can check when the city centre splits form the row houses.

In [ ]:
merged["cluster"] = cgram.labels[8].values
buildings["cluster"] = merged["cluster"]
buildings.plot(
    "cluster", categorical=True, figsize=(16, 16), legend=True
).set_axis_off()

## Prediction

Another application of this type of morphological data is prediction as morphology has power to predict certain urban phenomena. In this case, let's try to predict the index of multiple deprivation.

In [ ]:
imd = (
    geopandas.read_file(
        "https://github.com/martinfleis/gisruk2025/raw/refs/heads/main/imd.gpkg"
    ).set_index("lsoa11cd")
)
imd.head()

Let's focus on the most commonly reported value - rank.

In [ ]:
imd.explore("IMDRank0", tiles="cartodb positron")

We want to aggregate morphological data to LSOA. Get the LSOA ID on every building. 

In [ ]:
merged = merged.sjoin(imd[["geometry"]], how="left")

And group the data by LSOA, retrieving mean.

In [ ]:
grouper = merged.drop(
    columns=[
        "street_index",
        "node_index",
        "edge_index",
        "nodeID",
        "mm_len",
        "node_start",
        "node_end",
        "geometry",
    ]
).groupby("lsoa11cd")
lsoa_mean = grouper.mean()

This can be used as explanatory variables within a predictive model. We'll stick to simple linear regression, so standardization is due.

In [ ]:
standardized = (lsoa_mean - lsoa_mean.mean()) / lsoa_mean.std()

Train the model.

In [ ]:
lr = LinearRegression()
lr.fit(standardized, imd.IMDRank0)

We don't have to do train/test split as we are using regression (can't remember values as random forest might). So the performance can be measured directly on all of the data.

In [ ]:
lr.score(standardized, imd.IMDRank0)

$R^2$ of 0.77 is not bad considering we have measured just a small subset of morphometrics. Let's compare the prediction to the expected outcome.

In [ ]:
prediction = lr.predict(standardized)
residuals = imd.IMDRank0 - prediction

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 12))

max_red = residuals.abs().max()

imd.plot(
    "IMDRank0",
    legend=True,
    ax=ax[0],
    vmin=0,
    vmax=36000,
    legend_kwds={"shrink": 0.5, "orientation": "horizontal"},
)
imd.plot(
    prediction,
    legend=True,
    ax=ax[1],
    vmin=0,
    vmax=36000,
    legend_kwds={"shrink": 0.5, "orientation": "horizontal"},
)
imd.plot(
    residuals,
    vmin=-max_red,
    vmax=max_red,
    cmap="RdBu",
    legend=True,
    ax=ax[2],
    legend_kwds={"shrink": 0.5, "orientation": "horizontal"},
)

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

ax[0].set_title("IMDRank0")
ax[1].set_title("Predicted IMDRank0")
ax[2].set_title("Residuals")

It seems that the largest residuals are in areas which are next to those we predicted well. That may indicate spatial dependency in our data. Let's use it in our favour by including a spatial lag of our variables in the model. Build the contiguity graph.

In [ ]:
queen_lsoa = libpysal.graph.Graph.build_contiguity(imd, rook=False)

In [ ]:
m = imd.explore(tiles='cartodb positron')
queen_lsoa.explore(imd, m=m)

Measure lags for all of the variables.

In [ ]:
lags = {}
for col in standardized.columns:
    lags[f"{col}_lag"] = queen_lsoa.lag(standardized[col])

data_with_lag = pd.concat(
    [standardized, pd.DataFrame(lags, index=standardized.index)], axis=1
)

And fit a new model that includes lagged variables alongside original.

In [ ]:
lr_lag = LinearRegression()
lr_lag.fit(data_with_lag, imd.IMDRank0)

See the performance.

In [ ]:
lr_lag.score(data_with_lag, imd.IMDRank0)

0.9 is quite an improvement :). Let's visualise it, using the same color maps as above.

In [ ]:
prediction_lag = lr_lag.predict(data_with_lag)
residuals_lag = imd.IMDRank0 - prediction_lag

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 12))

imd.plot(
    "IMDRank0",
    legend=True,
    ax=ax[0],
    vmin=0,
    vmax=36000,
    legend_kwds={"shrink": 0.5, "orientation": "horizontal"},
)
imd.plot(
    prediction_lag,
    legend=True,
    ax=ax[1],
    vmin=0,
    vmax=36000,
    legend_kwds={"shrink": 0.5, "orientation": "horizontal"},
)
imd.plot(
    residuals_lag,
    vmin=-max_red,
    vmax=max_red,
    cmap="RdBu",
    legend=True,
    ax=ax[2],
    legend_kwds={"shrink": 0.5, "orientation": "horizontal"},
)

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

ax[0].set_title("IMDRank0")
ax[1].set_title("Predicted IMDRank0 (lagged)")
ax[2].set_title("Residuals (lagged)")

## Where to now?

We suggest spending some time in the documentation of [momepy](http://docs.momepy.org/), if you found this interesting.


----------

**Acknowledgements**

* Copyright: This manuscript has been authored in part by UT-Battelle, LLC under Contract No. DE-AC05-00OR22725 with the U.S. Department of Energy. The United States Government retains and the publisher, by accepting the article for publication, acknowledges that the United States Government retains a non-exclusive, paid-up, irrevocable, world-wide license to publish or reproduce the published form of this manuscript, or allow others to do so, for United States Government purposes. The Department of Energy will provide public access to these results of federally sponsored research in accordance with the DOE Public Access Plan (`http://energy.gov/downloads/doe-public-access-plan`).
